In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import requests
import re
from bs4 import BeautifulSoup
import json
import urllib


# отключить предупреждения пандас
pd.options.mode.chained_assignment = None

In [ ]:
# для запуска скрипта нужны файлы:
#     1 файл с транспортными регионами
#     2 файл с точками (центрами трнаспортных регионов)
# необходимо проставить дату и время!
# 

In [ ]:
# это дата для сохранения файлов
str_date = "{:%Y%m%d_%H%M}".format(datetime.now())

In [34]:
# дата, за которую нужно выгрузить маршруты
date_dep = '2020-08-26'
time_dep = '07:30:00'

In [3]:
transp_reg = gpd.read_file('../data/from/transp_reg_st.shp',
                        encoding='utf-8')
# transp_reg.crs = 'epsg:53004'
transp_reg = transp_reg.to_crs('epsg:4326')

In [4]:
gdf_pts_select = gpd.read_file('../data/res/gdf_pts_all_select.geojson',
                        encoding='utf-8')

In [5]:
# списки id районов (для составления пар)
lst_from = list(gdf_pts_select.reg_id.unique())
# lst_to = [532] * len(lst_from)
lst_to = list(gdf_pts_select.reg_id.unique())

In [6]:
# нахождение времен пешей доступности до/от центров районов, исходя из их площади
reg_area = transp_reg[['NO', 'geometry']].copy()
reg_area = reg_area.to_crs('epsg:20007')

lst_area=[]
i=0
for i in tqdm(range(len(reg_area))):
    area=round(reg_area.geometry[i].area / 1000 / 1000, 0)
    lst_area.append(area)
# 

reg_area['area_km'] = lst_area

reg_area = reg_area.sort_values(by=['area_km']).reset_index(drop=True)
reg_area['mint_ped'] = None

# проставление времен пешей досутпности
reg_area.mint_ped[:24] = 10
reg_area.mint_ped[24:47] = 15
reg_area.mint_ped[47:70] = 20
reg_area.mint_ped[70:93] = 25
reg_area.mint_ped[93:] = 30

reg_area = reg_area.sort_values(by=['NO']).reset_index(drop=True)
reg_area = reg_area.to_crs('epsg:4326')

In [7]:
def find_js(lnk):
    with requests.get(lnk, stream=True, timeout=25) as req:
        bs = BeautifulSoup(req.text, 'html.parser')
        all_scrpt = bs.find_all("script")
        for scr in all_scrpt:
            if scr.get('type') == "application/json":
                sc_ind = all_scrpt.index(scr)
                break
            else:
                sc_ind=0
        # 
        scrp_txt = all_scrpt[sc_ind].text
        js = json.loads(scrp_txt)

        return js

In [40]:
def get_dist_n_time(js, tp_ts, ind_min):

    if tp_ts == 'auto':
        # прибавить +5 +5 к времени для каждого конца, на парковку
        distnc = round(js['routerResponse']['routes'][ind_min]['distance']['value'] / 1000, 2)
        tm_avg = round(js['routerResponse']['routes'][ind_min]['duration'] / 60, 1) + 10
        tm_trfc = round(js['routerResponse']['routes'][ind_min]['durationInTraffic'] / 60, 1) + 10
    else:
        try:
            tm_avg = round(js['routerResponse']['routes'][ind_min]['duration'] / 60, 1)
        except:
            tm_avg = round(js['routerResponse']['routes'][0]['duration'] / 60, 1)
        distnc = None
        tm_trfc = None
    # 
    
    lst_dist_time = [distnc, tm_avg, tm_trfc]

    return lst_dist_time

In [9]:
def make_str_coords(one_reg_pt, two_reg_pt):
    coord_str = str(str(one_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(one_reg_pt.coords[0][0]) 
                    + "~" 
                    + str(two_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(two_reg_pt.coords[0][0]))
    # 
    return coord_str

In [10]:
def make_str_coords2(one_reg_pt, two_reg_pt):
    middle_pt = '53.438723%2C50.109816'
    coord_str = str(str(one_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(one_reg_pt.coords[0][0]) 
                    + "~" 
                    + middle_pt
                    + "~" 
                    + str(two_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(two_reg_pt.coords[0][0]))
    # 
    return coord_str

In [11]:
def get_rt_data(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to, ind_rt):

    coord_str = make_str_coords(one_reg_pt, two_reg_pt)
    str_coord = str(one_reg_pt.coords[0][1]) + "%2C"  +  str(one_reg_pt.coords[0][0])
    
    # tp_ts_mt="mt"
#     url_mt = "https://yandex.ru/maps/51/samara/?ll={}&mode=routes&rtext={}&rtt=mt&ruri=~&z=14.41". format(str_coord, coord_str)
    # tp_ts_auto = "auto"
    url_auto = "https://yandex.ru/maps/51/samara/?ll={}&mode=routes&rtext={}&rtt=auto&ruri=~&z=14.41". format(str_coord, coord_str)
    
    
#     js_mt = find_js(url_mt)
    js_auto = find_js(url_auto)
    
    # select shortest path
    rts_all = js['routerResponse']['routes']
    distnc_min = 1000000000
    ind_min = 0
    i=0
    for i in range(len(rts_all)):
        dstnc = js['routerResponse']['routes'][i]['distance']
        if dstnc < distnc_min:
            distnc_min = dstnc
            ind_min = i
    # 
    
    lst_dist_time_auto = get_dist_n_time(js_auto, 'auto', ind_min)
#     try:
#         lst_dist_time_mt = get_dist_n_time(js_mt, 'mt')
#     except:
#         lst_dist_time_mt = [None, None, None]
    x_from = one_reg_pt.x
    y_from = one_reg_pt.y
    x_to = two_reg_pt.x
    y_to = two_reg_pt.y
    
    line_geo = LineString(js_auto['routerResponse']['routes'][ind_min]['coordinates'])
    
    lst_one_data = [[reg_id_from, reg_id_to] 
                    + lst_dist_time_auto 
                    + [lst_dist_time_mt[1]] + [x_from, y_from, x_to, y_to] + [line_geo]]
    #
    return lst_one_data

In [12]:
def select_ind_rt(js_mt):
    rts = js_mt['routerResponse']['routes'][0]['paths']

    ind_rt = 0
    i=0
    for i in range(len(rts)):
        if len(rts[i]['segments']) > 1:
            j=0
            for j in range(len(rts[i]['segments'])):
                if rts[i]['segments'][j]['type'] == 'transport':
                    ind_rt = i
                    break
        if ind_rt > 0:
            break
    # 
    return ind_rt

In [16]:
def get_mt_rt_seq(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to, js_mt):
    
    ind_rt = select_ind_rt(js_mt)
    rt_segm = js_mt['routerResponse']['routes'][0]['paths'][ind_rt]['segments']

    lst_tp_seq = []
    cnt=1
    i=0
    for i in (range(len(rt_segm))):
        if rt_segm[i]['type'] == 'transport':
            one_rt = rt_segm[i]['transports'][0]
            line_id = one_rt['id']
            rt_type = one_rt['type']
            rt_name = one_rt['name']
            distnc = round(rt_segm[i]['distance']['value'] / 1000, 2)
            lst_tp_seq.append([reg_id_from, reg_id_to, cnt, line_id, rt_type, rt_name, distnc])
            cnt+=1
    #
    return lst_tp_seq

In [13]:
def get_mt_rt_time(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to, var_foo, mnt_from, mnt_to, date_dep, time_dep):
    
    if var_foo == 2:
        coord_str = make_str_coords2(one_reg_pt, two_reg_pt)
    else:
        coord_str = make_str_coords(one_reg_pt, two_reg_pt)
    
    str_coord = str(one_reg_pt.coords[0][0]) + "%2C" + str(one_reg_pt.coords[0][1])
    # исключение водного и жд транспорта, почти не работает)), 
    # если нет альтернатив, Яндекс все равно строит через водный или жд
    avoid_tp = '&routes%5BavoidTypes%5D=railway%2Cwater'

    # date_dep = '2020-08-24'
    # time_dep = '07:30:00'
    
    time_start = str('&routes%5BtimeDependent%5D%5Btime%5D={}T{}'.format(date_dep, time_dep) 
                     + '&routes%5BtimeDependent%5D%5Btype%5D=departure')
    # tp_ts_mt="mt"
    url_mt = "https://yandex.ru/maps/51/samara/?ll={}&mode=routes{}{}&rtext={}&rtt=mt&ruri=~&z=14.41". format(str_coord, avoid_tp, time_start, coord_str)

    js_mt = find_js(url_mt)

    ind_rt = select_ind_rt(js_mt)
    rt_segm = js_mt['routerResponse']['routes'][0]['paths'][ind_rt]['segments']
    #     lst_dist_time_mt = get_dist_n_time(js_mt, 'mt', ind_rt)
    
    # кол-во пересадок 
    trnsfer_cnt = js_mt['routerResponse']['routes'][0]['transfersCount'] + 1
    # плюс 1 это время для посадки на первый транспорт
    # ожидание каждой посадки - 5 минут
    time_wait_in_transfer = trnsfer_cnt * 5

    sum_clear_time = 0
    i=0
    for i in range(len(rt_segm)):
        # не прибавляем первое и последнее пешее время, 
        # так как оно усредненно посчитано для района
        # в Яндексе время в секундах!!
        if ((i != 0) & (i != (len(rt_segm) - 1))):
            segm_time = rt_segm[i]['duration']['value']
            sum_clear_time = sum_clear_time + segm_time
    # 
    sum_clear_time_all = round(sum_clear_time / 60, 1) + (mnt_from + mnt_to + time_wait_in_transfer)
    
    lst_dist_time_mt = [sum_clear_time_all, mnt_from, mnt_to, trnsfer_cnt]
    lst_tp_seq = get_mt_rt_seq(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to, js_mt)
    
    return lst_dist_time_mt, lst_tp_seq

In [14]:
def total_get_mt_data(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to, var_foo, mnt_from, mnt_to, date_dep, time_dep):
    try:
        lst_dist_time_mt,lst_tp_seq = get_mt_rt_time(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to, var_foo, mnt_from, mnt_to, date_dep, time_dep)
    except:
        time.sleep(30)
        try:
            lst_dist_time_mt,lst_tp_seq = get_mt_rt_time(one_reg_pt, two_reg_pt, reg_id_from, reg_id_to, var_foo, mnt_from, mnt_to, date_dep, time_dep)
        except:
            print(reg_id_from,reg_id_to)
            lst_dist_time_mt=0
            lst_tp_seq=0
            pass
    try:
        lst_dist_time_mt2,lst_tp_seq2 = get_mt_rt_time(two_reg_pt, one_reg_pt, reg_id_to, reg_id_from, var_foo, mnt_to, mnt_from, date_dep, time_dep)
    except:
        time.sleep(30)
        try:
            lst_dist_time_mt2,lst_tp_seq2 = get_mt_rt_time(two_reg_pt, one_reg_pt, reg_id_to, reg_id_from, var_foo, mnt_to, mnt_from, date_dep, time_dep)
        except:
            print(reg_id_to, reg_id_from)
            lst_dist_time_mt2=0
            lst_tp_seq=0
            pass

    #
    return lst_dist_time_mt, lst_dist_time_mt2, lst_tp_seq, lst_tp_seq2

In [35]:
# в цикле выполняется поиск маршрутов по обеим парам (и от 1 до 2, и от 2 до 1)
# в данном случае район 532 - особенный, для него нужна дополнительная точка, 
# иначе Яндекс не ищет маршрут на общественном транспорте оттуда/туда
# для 115 районов - 13110 пар (без 115 пар "сам в себя"), 
# только общественный транспорт - 3-3.5 часа
# общественный и авто - 8-10 часов


lst_total_data=[]
lst_time_data = []
lst_seq_data = []
lst_done=[]


j=0
# for j in tqdm(range(1)):
for j in tqdm(range(len(lst_from))):

    i=0
#     for i in tqdm(range(2)):
    for i in tqdm(range(len(lst_to))):

        reg_id_from, reg_id_to = lst_from[i], lst_to[j]
        if ((reg_id_from == 532) | (reg_id_to == 532)):
            var_foo = 2
        else:
            var_foo = 1
        #
        dnpr = str(reg_id_from) + "_" + str(reg_id_to)
        if dnpr not in lst_done:
            lst_done.append(dnpr)
            if (reg_id_from != reg_id_to):
                
                one_reg_pt = gdf_pts_select[gdf_pts_select.reg_id 
                                            == reg_id_from].reset_index(drop=True).geometry[0]
                two_reg_pt = gdf_pts_select[gdf_pts_select.reg_id 
                                            == reg_id_to].reset_index(drop=True).geometry[0]
                mnt_from = list(reg_area[reg_area.NO == reg_id_from]['mint_ped'])[0]
                mnt_to = list(reg_area[reg_area.NO == reg_id_to]['mint_ped'])[0]
                
                lst_dist_time_mt, lst_dist_time_mt2, lst_tp_seq, lst_tp_seq2 = total_get_mt_data(one_reg_pt, two_reg_pt, 
                                                                                                 reg_id_from, reg_id_to, 
                                                                                                 var_foo, 
                                                                                                 mnt_from, mnt_to, 
                                                                                                 date_dep, time_dep)
                #
                if lst_dist_time_mt != 0:
                    lst_time_data.append([reg_id_from, reg_id_to] + lst_dist_time_mt)
                # 
                if lst_dist_time_mt2 != 0:
                    lst_time_data.append([reg_id_to, reg_id_from] + lst_dist_time_mt2)
                # 
                if lst_tp_seq != 0:
                    lst_seq_data = lst_seq_data + lst_tp_seq
                # 
                if lst_tp_seq2 != 0:
                    lst_seq_data = lst_seq_data + lst_tp_seq2
                # 
# 

In [36]:
clmns=['id_reg_from', 'id_reg_to', 'seq_in_rt', 'line_id', 
       'tp_ts', 'name', 'distance']
df_rt_seq = pd.DataFrame(lst_seq_data, columns = clmns)

In [37]:
df_rt_seq.to_csv('./res/df_rt_seq_{}.csv'.format(str_date), sep=';', 
                 encoding='utf-8-sig', index=False)

In [38]:
clmns2 = ['id_reg_from', 'id_reg_to','time_pubtr2', 'mnt_from_reg', 'mnt_to_reg', 'cnt_transprt']
df_time = pd.DataFrame(lst_time_data, columns=clmns2)
df_time = df_time.drop_duplicates(subset=['id_reg_from', 'id_reg_to']).reset_index(drop=True)

In [39]:
df_time.to_csv('./res/df_rt_time_clear_{}.csv'.format(str_date), sep=';', 
               encoding='utf-8-sig', index=False)

In [41]:
gdf_rt_shorter_all = gpd.read_file('../data/res/big_ok/gdf_rt_shorter_all.json',
                        encoding='utf-8')

In [42]:
#  временное прибавление по 5 минут на каждом конце
gdf_rt_shorter_all['time_auto_avg'] = gdf_rt_shorter_all['time_auto_avg'] + 10
gdf_rt_shorter_all['time_auto_traff'] = gdf_rt_shorter_all['time_auto_traff'] + 10

In [51]:
gdf_rt_shorter_all4 = gdf_rt_shorter_all.merge(df_time, how='left', 
                                on = ['id_reg_from', 'id_reg_to'])

In [52]:
gdf_rt_shorter_all4.time_pubtr = gdf_rt_shorter_all4.time_pubtr2

In [53]:
del gdf_rt_shorter_all4['time_pubtr2']

In [54]:
ntgeo_gdf_rt_all4 = gdf_rt_shorter_all4.copy()
del ntgeo_gdf_rt_all4['geometry']
ntgeo_gdf_rt_all4.to_csv('../data/res/big_ok/matrice_len_time_csv.csv', 
               sep=';', encoding='utf-8-sig', index=False)

In [55]:
gdf_rt_shorter_all4.to_file('../data/res/big_ok/matrice_len_time.json', driver='GeoJSON', 
                            encoding='utf-8')

In [56]:
gdf_rt_shorter_all4.columns

Index(['index', 'id_reg_from', 'id_reg_to', 'length', 'time_auto_avg',
       'time_auto_traff', 'time_pubtr', 'x_from', 'y_from', 'x_to', 'y_to',
       'geometry', 'mnt_from_reg', 'mnt_to_reg', 'cnt_transprt'],
      dtype='object')